## CNN
- 앞의 Dense 예제에서는 이미지의 공간 구조나 관계 정보를 사용하지 않았음 (flatten화 해서 다 무시)
- 사람이 무언가를 보고 인식할 때, 여러 단계를 거쳐서 각 단꼐마다 점점 더 구조화된 정보 인식
- **책의 설명은 이해할 수 없는 수준**이므로 다 무시하고 이론은 각자 공부하시오

## LeNet 구현
- keras.layers.convolutional.**Conv2D**(filters, kernel_size, padding='valid')
    - padding='same' ; 패딩을 사용함
    - padding='valid' ; 입력과 필터가 완전히 겹치는 위치에서만 컨볼루션 계산 (출력 작아짐)
- keras.layers.pooling.**MaxPooling2D**(pool_size(2, 2), strides=(2, 2))
- *Convolutional Networks for Images, Speech, and Time-Series, brain theory neural networks, vol. 3361, 1995*

In [1]:
from keras import backend as K
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Activation, Flatten, Dense
from keras.datasets import mnist
from keras.utils import np_utils
from keras.optimizers import SGD, RMSprop, Adam
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
class LeNet:
    @staticmethod
    def build(input_shape, classes):
        model = Sequential()
        model.add(Conv2D(20, kernel_size=5, padding="same", input_shape=input_shape))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size(2,2), strides))